In [46]:
import pandas as pd
import numpy as np

project_name = "Aug_8"

import os
if not os.path.exists(f'{project_name}/control_results'):
    os.makedirs(f'{project_name}/control_results')

In [47]:
log = pd.read_csv(f'../log/{project_name}/koopman_results_log.csv')
log

,env_name,state_dim,u_dim,train_samples,val_samples,test_samples,Ksteps,train_steps,batch_size,initial_lr,...,normalize,best_val_Kloss,convergence_val_Kloss,test_Kloss,test_CovLoss,test_ControlLoss,model_path,num_params,encoder_num_params,m
0,DampingPendulum,2,1.0,60000,20000,20000,15,60000,64,0.001,...,False,0.012904,0.012916,0.012920,1.172993e+07,1413.839111,../log/Aug_8/best_models/20250808_010058_model...,134168,134148,100
1,DampingPendulum,2,1.0,60000,20000,20000,15,60000,64,0.001,...,False,0.028970,0.030188,0.029028,7.072261e-06,69.266296,../log/Aug_8/best_models/20250808_010403_model...,134168,134148,100
2,DampingPendulum,2,1.0,60000,20000,20000,15,60000,64,0.001,...,False,0.075336,0.075563,0.075561,6.422688e+00,21.263502,../log/Aug_8/best_models/20250808_010719_model...,134168,134148,100
3,DampingPendulum,2,1.0,60000,20000,20000,15,60000,64,0.001,...,False,0.093591,0.093697,0.093777,7.106797e-05,21.263470,../log/Aug_8/best_models/20250808_011116_model...,134168,134148,100
4,DampingPendulum,2,1.0,60000,20000,20000,15,60000,64,0.001,...,False,0.011116,0.011133,0.011115,2.938835e+04,2972.479736,../log/Aug_8/best_models/20250808_011517_model...,135218,135176,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3041,Polynomial,3,NaN,4000,20000,20000,1,80000,64,0.001,...,False,0.164632,0.169592,0.179662,2.632077e+03,NaN,../log/Aug_8/best_models/20250921_171349_model...,135210,135174,200
3042,Polynomial,3,NaN,16000,20000,20000,1,80000,64,0.001,...,False,0.021344,0.023537,0.021194,3.167876e+03,NaN,../log/Aug_8/best_models/20250921_171526_model...,135210,135174,200
3043,Polynomial,3,NaN,60000,20000,20000,1,80000,64,0.001,...,False,0.015689,0.017325,0.011621,2.449296e+03,NaN,../log/Aug_8/best_models/20250921_171714_model...,135210,135174,200
3044,Polynomial,3,NaN,1000,20000,20000,1,80000,64,0.001,...,False,1.281956,1.293628,1.374917,4.306345e-05,NaN,../log/Aug_8/best_models/20250921_171850_model...,135210,135174,200


In [3]:
franka_results = pd.read_csv(f'../log/{project_name}/franka_control_results.csv')
franka_results

,env_name,seed,train_samples,encode_dim,use_control_loss,use_covariance_loss,max_eig,error_eight,error_star
0,Franka,17382,60000,17,False,False,1.000523,1.982986,2.544477
1,Franka,17382,60000,17,False,True,1.001274,3.384295,2.909771
2,Franka,17382,60000,17,True,False,1.000684,2.570720,2.180488
3,Franka,17382,60000,17,True,True,1.000466,1.981111,1.692435
4,Franka,17382,60000,34,False,False,1.001324,19.011537,17.723866
...,...,...,...,...,...,...,...,...,...
395,Franka,51194,4000,272,True,False,0.975590,31.877749,30.758160
396,Franka,51194,16000,272,True,False,0.940928,18.878293,32.632786
397,Franka,51194,1000,272,True,True,1.029612,1.817901,1.525587
398,Franka,51194,4000,272,True,True,0.991349,32.163100,14.040351


In [4]:
grouped = franka_results.groupby(['env_name', 'encode_dim', 'train_samples', 'use_control_loss', 'use_covariance_loss']).agg({
    'error_eight': ['mean', 'std'],
    'error_star': ['mean', 'std']
}).reset_index().sort_values(by=['train_samples', 'encode_dim'])
grouped.to_csv(f'{project_name}/control_results/Franka.csv', index=False)

In [5]:
grouped

env_name encode_dim train_samples use_control_loss use_covariance_loss  \
                                                                            
0    Franka         17          1000            False               False   
1    Franka         17          1000            False                True   
2    Franka         17          1000             True               False   
3    Franka         17          1000             True                True   
16   Franka         34          1000            False               False   
..      ...        ...           ...              ...                 ...   
63   Franka        136         60000             True                True   
76   Franka        272         60000            False               False   
77   Franka        272         60000            False                True   
78   Franka        272         60000             True               False   
79   Franka        272         60000             True                True   

   error_eight            error_star             
          mean        std       mean        std  
0     2.107360   1.052512   1.925200   0.922419  
1     1.880517   0.329411   1.815664   0.337805  
2     6.441222  10.658306   6.911326  12.097212  
3     1.839730   0.357825   1.666700   0.191181  
16    6.891838  12.271509   6.305510  10.774318  
..         ...        ...        ...        ...  
63   11.638516  17.235904  11.867956  14.203790  
76    2.426034   2.646393   2.016051   1.949919  
77    1.873710   0.750318   1.659628   0.469429  
78    1.836688   1.118414   1.554135   0.744937  
79    1.824505   0.424397   1.646200   0.279311  

[80 rows x 9 columns]

In [6]:
pendulum_results = pd.read_csv(f'../log/{project_name}/pendulum_control_results.csv')
pendulum_results

,env_name,encode_dim,use_control_loss,use_covariance_loss,train_samples,n_models,n_stable,stability_rate,finite_best_count,median_best_last10_L1,median_best_full_L1,median_best_control_cost,mean_best_last10_L1,mean_best_full_L1,mean_best_control_cost
0,DampingPendulum,2,False,False,1000,5,5,1.0,5,0.839624,1.658373,6838.069676,0.827942,1.952128,30855.017223
1,DampingPendulum,2,False,False,4000,5,5,1.0,5,0.745876,1.770350,5775.178679,0.608533,2.877944,41054.781140
2,DampingPendulum,2,False,False,16000,5,5,1.0,5,1.041948,1.209062,533.129206,0.749193,2.434110,31042.507158
3,DampingPendulum,2,False,False,60000,5,5,1.0,5,1.060279,1.212843,537.322711,0.746100,1.663338,19297.884580
4,DampingPendulum,2,False,True,1000,5,4,0.8,4,1.125503,1.497966,364.922519,1.093608,1.524554,550.836131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,DoublePendulum,64,True,False,60000,5,2,0.4,2,1.906815,1.512023,8776.158235,1.906815,1.512023,8776.158235
156,DoublePendulum,64,True,True,1000,5,5,1.0,5,0.918572,1.009794,5168.580074,1.325567,1.461312,7925.013301
157,DoublePendulum,64,True,True,4000,5,5,1.0,5,0.872281,1.033193,20210.984095,0.915613,1.286920,39398.416365
158,DoublePendulum,64,True,True,16000,5,3,0.6,3,1.219856,1.183453,2321.832304,1.419414,1.556418,55228.674958


In [7]:
grouped = pendulum_results[['env_name', 'encode_dim', 'train_samples', 'use_control_loss', 'use_covariance_loss', 'stability_rate', 'median_best_last10_L1', 'median_best_control_cost']]

# change column name from median_best_last10_L1 to error
grouped = grouped.rename(columns={"median_best_last10_L1": "error", 'median_best_control_cost': 'control_cost'}).sort_values(by=['train_samples', 'encode_dim'])
grouped[grouped['env_name'] == 'DampingPendulum'].to_csv(f'{project_name}/control_results/DampingPendulum.csv', index=False)
grouped[grouped['env_name'] == 'DoublePendulum'].to_csv(f'{project_name}/control_results/DoublePendulum.csv', index=False)

In [8]:
grouped[grouped['env_name'] == 'DampingPendulum']

,env_name,encode_dim,train_samples,use_control_loss,use_covariance_loss,stability_rate,error,control_cost
0,DampingPendulum,2,1000,False,False,1.0,0.839624,6838.069676
4,DampingPendulum,2,1000,False,True,0.8,1.125503,364.922519
8,DampingPendulum,2,1000,True,False,1.0,1.151157,26350.914543
12,DampingPendulum,2,1000,True,True,1.0,0.777051,3464.542677
16,DampingPendulum,4,1000,False,False,1.0,1.036375,500.798782
...,...,...,...,...,...,...,...,...
63,DampingPendulum,16,60000,True,True,1.0,0.531944,22699.940799
67,DampingPendulum,32,60000,False,False,1.0,0.201702,1234.264073
71,DampingPendulum,32,60000,False,True,1.0,1.018207,519.011944
75,DampingPendulum,32,60000,True,False,1.0,0.014481,15530.342954


In [9]:
grouped[grouped['env_name'] == 'DoublePendulum']

,env_name,encode_dim,train_samples,use_control_loss,use_covariance_loss,stability_rate,error,control_cost
80,DoublePendulum,4,1000,False,False,1.0,0.493794,5981.731117
84,DoublePendulum,4,1000,False,True,1.0,0.311326,694.057666
88,DoublePendulum,4,1000,True,False,0.6,0.589538,4281.151189
92,DoublePendulum,4,1000,True,True,1.0,1.350587,1923.512407
96,DoublePendulum,8,1000,False,False,0.8,0.296663,1795.908015
...,...,...,...,...,...,...,...,...
143,DoublePendulum,32,60000,True,True,1.0,1.292997,1903.287333
147,DoublePendulum,64,60000,False,False,0.8,0.155232,846.488326
151,DoublePendulum,64,60000,False,True,1.0,0.674094,742.706018
155,DoublePendulum,64,60000,True,False,0.4,1.906815,8776.158235


In [54]:
isaacsim_results = pd.read_csv(f'../log/{project_name}/isaac_control_results.csv')
isaacsim_results = isaacsim_results.merge(log[['env_name', 'model_path', 'encode_dim', 'use_control_loss', 'use_covariance_loss', 'seed', 'train_samples']], on='model_path', how='left').drop(columns=['model_path', 'robot'])[['env_name','seed', 'train_samples', 'encode_dim', 'use_control_loss', 'use_covariance_loss', 'mean_JrPE', 'mean_survival_steps']].sort_values(by=['env_name', 'use_control_loss', 'use_covariance_loss'], ascending=[True, True, True])
isaacsim_results

,env_name,seed,train_samples,encode_dim,use_control_loss,use_covariance_loss,mean_JrPE,mean_survival_steps
0,G1,17382,200000,53,False,False,0.148349,53.688
4,G1,17382,200000,106,False,False,0.167348,55.219
8,G1,17382,200000,212,False,False,0.177962,58.296
12,G1,17382,200000,424,False,False,0.156307,58.436
16,G1,17382,200000,848,False,False,0.142367,60.102
...,...,...,...,...,...,...,...,...
786,Go2,51194,9333,280,True,True,0.668062,2.623
787,Go2,51194,37333,280,True,True,0.159144,96.116
797,Go2,51194,2333,560,True,True,0.619318,2.483
798,Go2,51194,9333,560,True,True,0.698119,2.742


In [55]:
grouped = isaacsim_results.groupby(['env_name', 'train_samples', 'encode_dim', 'use_control_loss', 'use_covariance_loss']).agg({
    'mean_JrPE': ['mean', 'std'],
    'mean_survival_steps': ['mean', 'std']
}).reset_index()
grouped[grouped['env_name'] == 'G1'].to_csv(f'{project_name}/control_results/G1.csv', index=False)
grouped[grouped['env_name'] == 'Go2'].to_csv(f'{project_name}/control_results/Go2.csv', index=False)

In [56]:
grouped

env_name train_samples encode_dim use_control_loss use_covariance_loss  \
                                                                             
0         G1          3333         53            False               False   
1         G1          3333         53            False                True   
2         G1          3333         53             True               False   
3         G1          3333         53             True                True   
4         G1          3333        106            False               False   
..       ...           ...        ...              ...                 ...   
155      Go2        140000        280             True                True   
156      Go2        140000        560            False               False   
157      Go2        140000        560            False                True   
158      Go2        140000        560             True               False   
159      Go2        140000        560             True                True   

    mean_JrPE           mean_survival_steps             
         mean       std                mean        std  
0    0.279228  0.047452              5.9154   0.812213  
1    0.279228  0.047452              5.9154   0.812213  
2    0.256019  0.026993             10.8048   1.749558  
3    0.256019  0.026993             10.8048   1.749558  
4    0.315275  0.031284              5.2890   0.722510  
..        ...       ...                 ...        ...  
155  0.126268  0.016136            101.1064  10.892510  
156  0.164964  0.009751             68.2394  10.873705  
157  0.162613  0.021591             64.2830  11.157816  
158  0.136721  0.016523             80.6608  16.914025  
159  0.124954  0.014971             95.9902   8.257797  

[160 rows x 9 columns]

In [58]:
# ======================================================
# CONTROL RESULTS — COMBINED (no shaded fill, bug-fixed)
#   • Avg over all z  → plots vs train_samples (4 lines)
#   • Avg over samples → plots vs encode_dim (4 lines)
#   • Tables with % improvements vs baseline (+ synergy)
# ======================================================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

out_dir = os.path.join(project_name, "control_results_visualization")
os.makedirs(out_dir, exist_ok=True)

# ---------- Flatten MultiIndex columns & normalize names ----------
def _flatten_grouped(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ["_".join([str(c) for c in tup if c not in ("", None)]).strip("_")
                      for tup in df.columns.values]
    rename = {
        "mean_JrPE_mean": "JrPE",
        "mean_JrPE_std": "JrPE_std",
        "mean_survival_steps_mean": "Survival",
        "mean_survival_steps_std": "Survival_std",
        "use_covariance_loss": "cov",
        "use_control_loss": "ctrl",
        "env_name": "env_name",
        "train_samples": "train_samples",
        "encode_dim": "encode_dim",
    }
    df.rename(columns={k:v for k,v in rename.items() if k in df.columns}, inplace=True)
    for b in ("cov", "ctrl"):
        if b in df.columns and df[b].dtype != bool:
            df[b] = df[b].astype(bool)
    return df

GG = _flatten_grouped(grouped)

# ---------- Aggregations ----------
# A) Average over ALL encode_dim (equal weight) → lines vs train_samples
AggZ = (GG
        .groupby(["env_name","train_samples","cov","ctrl"], as_index=False)
        .agg(JrPE=("JrPE","mean"),
             Survival=("Survival","mean"),
             n_z=("encode_dim","nunique"))
        .sort_values(["env_name","train_samples","cov","ctrl"])
       )

# B) Average over ALL train_samples (equal weight) → lines vs encode_dim
AggTS = (GG
         .groupby(["env_name","encode_dim","cov","ctrl"], as_index=False)
         .agg(JrPE=("JrPE","mean"),
              Survival=("Survival","mean"),
              n_samples=("train_samples","nunique"))
         .sort_values(["env_name","encode_dim","cov","ctrl"])
        )

# ---------- Helpers ----------
def _pct_impr(new, base, higher_is_better: bool) -> float:
    if base is None or pd.isna(base) or base == 0: return np.nan
    if new  is None or pd.isna(new):               return np.nan
    return ((new - base)/base * 100.0) if higher_is_better else ((base - new)/base * 100.0)

def _effect_table_by_samples(Aenv: pd.DataFrame, env: str) -> pd.DataFrame:
    """Per train_samples at this env, avg over z: abs metrics + % improvements vs baseline."""
    rows = []
    for ts, g in Aenv.groupby("train_samples"):
        def pick(cov, ctrl, col):
            s = g[(g["cov"]==cov) & (g["ctrl"]==ctrl)]
            return float(s[col].iloc[0]) if not s.empty and col in s.columns else np.nan
        jrpe_b   = pick(False, False, "JrPE");     surv_b   = pick(False, False, "Survival")
        jrpe_cov = pick(True,  False, "JrPE");     surv_cov = pick(True,  False, "Survival")
        jrpe_ctl = pick(False, True,  "JrPE");     surv_ctl = pick(False, True,  "Survival")
        jrpe_bth = pick(True,  True,  "JrPE");     surv_bth = pick(True,  True,  "Survival")
        row = {
            "Environment": env, "train_samples": ts, "n_z_avgd": int(g["n_z"].max()),
            "JrPE_base": jrpe_b, "JrPE_cov": jrpe_cov, "JrPE_ctrl": jrpe_ctl, "JrPE_both": jrpe_bth,
            "Surv_base": surv_b, "Surv_cov": surv_cov, "Surv_ctrl": surv_ctl, "Surv_both": surv_bth,
            "JrPE_impr_cov_%":  _pct_impr(jrpe_cov, jrpe_b, higher_is_better=False),
            "JrPE_impr_ctrl_%": _pct_impr(jrpe_ctl, jrpe_b, higher_is_better=False),
            "JrPE_impr_both_%": _pct_impr(jrpe_bth, jrpe_b, higher_is_better=False),
            "Surv_impr_cov_%":  _pct_impr(surv_cov,  surv_b, higher_is_better=True),
            "Surv_impr_ctrl_%": _pct_impr(surv_ctl,  surv_b, higher_is_better=True),
            "Surv_impr_both_%": _pct_impr(surv_bth,  surv_b, higher_is_better=True),
        }
        row["JrPE_synergy_%"] = row["JrPE_impr_both_%"] - np.nanmax([row["JrPE_impr_cov_%"], row["JrPE_impr_ctrl_%"]])
        row["Surv_synergy_%"] = row["Surv_impr_both_%"] - np.nanmax([row["Surv_impr_cov_%"], row["Surv_impr_ctrl_%"]])
        rows.append(row)
    T = pd.DataFrame(rows).sort_values("train_samples")
    pct_cols = [c for c in T.columns if c.endswith("_%")]
    for c in pct_cols: T[c] = T[c].round(1)
    return T

def _effect_table_by_z(Aenv: pd.DataFrame, env: str) -> pd.DataFrame:
    """Per encode_dim at this env, avg over samples: abs metrics + % improvements vs baseline."""
    rows = []
    for z, g in Aenv.groupby("encode_dim"):
        def pick(cov, ctrl, col):
            s = g[(g["cov"]==cov) & (g["ctrl"]==ctrl)]
            return float(s[col].iloc[0]) if not s.empty and col in s.columns else np.nan
        jrpe_b   = pick(False, False, "JrPE");     surv_b   = pick(False, False, "Survival")
        jrpe_cov = pick(True,  False, "JrPE");     surv_cov = pick(True,  False, "Survival")
        jrpe_ctl = pick(False, True,  "JrPE");     surv_ctl = pick(False, True,  "Survival")
        jrpe_bth = pick(True,  True,  "JrPE");     surv_bth = pick(True,  True,  "Survival")
        row = {
            "Environment": env, "encode_dim": int(z), "n_samples_avgd": int(g["n_samples"].max()),
            "JrPE_base": jrpe_b, "JrPE_cov": jrpe_cov, "JrPE_ctrl": jrpe_ctl, "JrPE_both": jrpe_bth,
            "Surv_base": surv_b, "Surv_cov": surv_cov, "Surv_ctrl": surv_ctl, "Surv_both": surv_bth,
            "JrPE_impr_cov_%":  _pct_impr(jrpe_cov, jrpe_b, higher_is_better=False),
            "JrPE_impr_ctrl_%": _pct_impr(jrpe_ctl, jrpe_b, higher_is_better=False),
            "JrPE_impr_both_%": _pct_impr(jrpe_bth, jrpe_b, higher_is_better=False),
            "Surv_impr_cov_%":  _pct_impr(surv_cov,  surv_b, higher_is_better=True),
            "Surv_impr_ctrl_%": _pct_impr(surv_ctl,  surv_b, higher_is_better=True),
            "Surv_impr_both_%": _pct_impr(surv_bth,  surv_b, higher_is_better=True),
        }
        row["JrPE_synergy_%"] = row["JrPE_impr_both_%"] - np.nanmax([row["JrPE_impr_cov_%"], row["JrPE_impr_ctrl_%"]])
        row["Surv_synergy_%"] = row["Surv_impr_both_%"] - np.nanmax([row["Surv_impr_cov_%"], row["Surv_impr_ctrl_%"]])
        rows.append(row)
    Tz = pd.DataFrame(rows).sort_values("encode_dim")
    pct_cols = [c for c in Tz.columns if c.endswith("_%")]
    for c in pct_cols: Tz[c] = Tz[c].round(1)
    return Tz

def _plot_four_lines(ax, df, x_col, y_col, title, xlabel, logx=False):
    order = [(False, False, "baseline"),
             (True,  False, "+cov"),
             (False, True,  "+ctrl"),
             (True,  True,  "+both")]
    for cov, ctrl, label in order:
        g = df[(df["cov"]==cov) & (df["ctrl"]==ctrl)].sort_values(x_col)
        if g.empty: 
            continue
        lw = 2.4 if (cov and ctrl) or (not cov and not ctrl) else 1.8
        ls = "-" if (cov and ctrl) or (not cov and not ctrl) else "--"
        ax.plot(g[x_col].values, g[y_col].values, marker="o", linewidth=lw, linestyle=ls, label=label)
    if logx:
        try:
            ax.set_xscale("log", base=2 if x_col=="encode_dim" else 10)
        except TypeError:
            ax.set_xscale("log", basex=2 if x_col=="encode_dim" else 10)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(y_col if y_col!="Survival" else "Survival steps")
    ax.set_title(title)
    ax.grid(True, which="both", alpha=0.4, ls="--")
    ax.legend(ncol=2, fontsize=9)

# ---------- Build plots + tables for each env ----------
env_list = ["G1", "Go2"]  # change if you want more envs included
summ_samples_rows = []
summ_z_rows = []

for env in env_list:
    # ===== Avg over ALL z → vs train_samples =====
    AenvZ = AggZ[AggZ["env_name"] == env].copy()
    if not AenvZ.empty:
        # tables
        T_samples = _effect_table_by_samples(AenvZ, env)
        p_csv = os.path.join(out_dir, f"{env}_effects_by_samples_avgZ.csv")
        T_samples.to_csv(p_csv, index=False); print(f"Saved: {p_csv}")
        # summary at max samples
        ts_max = T_samples["train_samples"].max()
        last = T_samples[T_samples["train_samples"] == ts_max].iloc[0]
        summ_samples_rows.append({
            "Environment": env, "train_samples": ts_max, "avg_over_z": True,
            "JrPE_base": last["JrPE_base"],
            "JrPE_impr_cov_%": last["JrPE_impr_cov_%"],
            "JrPE_impr_ctrl_%": last["JrPE_impr_ctrl_%"],
            "JrPE_impr_both_%": last["JrPE_impr_both_%"],
            "Surv_base": last["Surv_base"],
            "Surv_impr_cov_%": last["Surv_impr_cov_%"],
            "Surv_impr_ctrl_%": last["Surv_impr_ctrl_%"],
            "Surv_impr_both_%": last["Surv_impr_both_%"],
            "JrPE_synergy_%": last["JrPE_synergy_%"],
            "Surv_synergy_%": last["Surv_synergy_%"],
            "n_z_covered": int(AenvZ["n_z"].max())
        })
        # plots (no fill)
        fig, ax = plt.subplots(figsize=(7,5))
        _plot_four_lines(ax, AenvZ, x_col="train_samples", y_col="JrPE",
                         title=f"{env} — JrPE vs train_samples (avg over all z)",
                         xlabel="Train samples (log)", logx=True)
        fig.tight_layout()
        p1 = os.path.join(out_dir, f"{env}_JrPE_vs_samples_avgZ.png")
        plt.savefig(p1, dpi=300); plt.close(fig); print(f"Saved: {p1}")

        fig, ax = plt.subplots(figsize=(7,5))
        _plot_four_lines(ax, AenvZ, x_col="train_samples", y_col="Survival",
                         title=f"{env} — Survival vs train_samples (avg over all z)",
                         xlabel="Train samples (log)", logx=True)
        fig.tight_layout()
        p2 = os.path.join(out_dir, f"{env}_Survival_vs_samples_avgZ.png")
        plt.savefig(p2, dpi=300); plt.close(fig); print(f"Saved: {p2}")
    else:
        print(f"[combined] No rows for {env} in AggZ (avg over z).")

    # ===== Avg over ALL samples → vs encode_dim =====
    AenvTS = AggTS[AggTS["env_name"] == env].copy()
    if not AenvTS.empty:
        # tables
        T_z = _effect_table_by_z(AenvTS, env)
        p_csv = os.path.join(out_dir, f"{env}_effects_by_encode_dim_avgSamples.csv")
        T_z.to_csv(p_csv, index=False); print(f"Saved: {p_csv}")
        # summary at max z
        z_max = T_z["encode_dim"].max()
        last = T_z[T_z["encode_dim"] == z_max].iloc[0]
        summ_z_rows.append({
            "Environment": env, "encode_dim": int(z_max), "avg_over_samples": True,
            "JrPE_base": last["JrPE_base"],
            "JrPE_impr_cov_%": last["JrPE_impr_cov_%"],
            "JrPE_impr_ctrl_%": last["JrPE_impr_ctrl_%"],
            "JrPE_impr_both_%": last["JrPE_impr_both_%"],
            "Surv_base": last["Surv_base"],
            "Surv_impr_cov_%": last["Surv_impr_cov_%"],
            "Surv_impr_ctrl_%": last["Surv_impr_ctrl_%"],
            "Surv_impr_both_%": last["Surv_impr_both_%"],
            "JrPE_synergy_%": last["JrPE_synergy_%"],
            "Surv_synergy_%": last["Surv_synergy_%"],
            "n_samples_covered": int(AenvTS["n_samples"].max())
        })
        # plots (no fill)
        fig, ax = plt.subplots(figsize=(7,5))
        _plot_four_lines(ax, AenvTS, x_col="encode_dim", y_col="JrPE",
                         title=f"{env} — JrPE vs encode_dim (avg over all samples)",
                         xlabel="Encode dimension z (log₂)", logx=True)
        fig.tight_layout()
        p3 = os.path.join(out_dir, f"{env}_JrPE_vs_z_avgSamples.png")
        plt.savefig(p3, dpi=300); plt.close(fig); print(f"Saved: {p3}")

        fig, ax = plt.subplots(figsize=(7,5))
        _plot_four_lines(ax, AenvTS, x_col="encode_dim", y_col="Survival",
                         title=f"{env} — Survival vs encode_dim (avg over all samples)",
                         xlabel="Encode dimension z (log₂)", logx=True)
        fig.tight_layout()
        p4 = os.path.join(out_dir, f"{env}_Survival_vs_z_avgSamples.png")
        plt.savefig(p4, dpi=300); plt.close(fig); print(f"Saved: {p4}")
    else:
        print(f"[combined] No rows for {env} in AggTS (avg over samples).")

# ---------- Cross-env summaries ----------
if summ_samples_rows:
    S1 = pd.DataFrame(summ_samples_rows)
    pct_cols = [c for c in S1.columns if c.endswith("_%")]
    for c in pct_cols: S1[c] = S1[c].round(1)
    path1 = os.path.join(out_dir, "summary_avgZ_at_max_samples.csv")
    S1.to_csv(path1, index=False); print(f"Saved summary: {path1}")
    try: display(S1)
    except Exception: pass

if summ_z_rows:
    S2 = pd.DataFrame(summ_z_rows)
    pct_cols = [c for c in S2.columns if c.endswith("_%")]
    for c in pct_cols: S2[c] = S2[c].round(1)
    path2 = os.path.join(out_dir, "summary_avgSamples_at_maxZ.csv")
    S2.to_csv(path2, index=False); print(f"Saved summary: {path2}")
    try: display(S2)
    except Exception: pass

Saved: Aug_8/control_results_visualization/G1_effects_by_samples_avgZ.csv
Saved: Aug_8/control_results_visualization/G1_JrPE_vs_samples_avgZ.png
Saved: Aug_8/control_results_visualization/G1_Survival_vs_samples_avgZ.png
Saved: Aug_8/control_results_visualization/G1_effects_by_encode_dim_avgSamples.csv
Saved: Aug_8/control_results_visualization/G1_JrPE_vs_z_avgSamples.png
Saved: Aug_8/control_results_visualization/G1_Survival_vs_z_avgSamples.png
Saved: Aug_8/control_results_visualization/Go2_effects_by_samples_avgZ.csv
Saved: Aug_8/control_results_visualization/Go2_JrPE_vs_samples_avgZ.png
Saved: Aug_8/control_results_visualization/Go2_Survival_vs_samples_avgZ.png
Saved: Aug_8/control_results_visualization/Go2_effects_by_encode_dim_avgSamples.csv
Saved: Aug_8/control_results_visualization/Go2_JrPE_vs_z_avgSamples.png
Saved: Aug_8/control_results_visualization/Go2_Survival_vs_z_avgSamples.png
Saved summary: Aug_8/control_results_visualization/summary_avgZ_at_max_samples.csv


,Environment,train_samples,avg_over_z,JrPE_base,JrPE_impr_cov_%,JrPE_impr_ctrl_%,JrPE_impr_both_%,Surv_base,Surv_impr_cov_%,Surv_impr_ctrl_%,Surv_impr_both_%,JrPE_synergy_%,Surv_synergy_%,n_z_covered
0,G1,200000,True,0.149400,-5.3,7.0,2.7,58.74924,-0.2,-5.4,-4.2,-4.3,-4.1,5
1,Go2,140000,True,0.176546,5.7,26.5,26.9,67.35724,9.7,28.9,48.2,0.4,19.3,5


Saved summary: Aug_8/control_results_visualization/summary_avgSamples_at_maxZ.csv


,Environment,encode_dim,avg_over_samples,JrPE_base,JrPE_impr_cov_%,JrPE_impr_ctrl_%,JrPE_impr_both_%,Surv_base,Surv_impr_cov_%,Surv_impr_ctrl_%,Surv_impr_both_%,JrPE_synergy_%,Surv_synergy_%,n_samples_covered
0,G1,848,True,0.224268,1.3,33.7,32.0,31.63770,1.0,34.0,35.1,-1.7,1.1,4
1,Go2,560,True,0.401803,1.0,0.6,3.6,24.52495,2.3,59.1,93.0,2.6,33.8,4
